In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
import time
import math
from bs4 import BeautifulSoup


In [2]:
baseUrl = 'https://api.llama.fi'

protocols = requests.get(baseUrl + '/protocols')

In [3]:
protocols_data = protocols.json()

extracted_data = []

# Extraer los datos específicos para cada protocolo
for protocol in protocols_data:
    protocol_info = {
        'name': protocol.get('name'),                    # Nombre del protocolo
        'symbol': protocol.get('symbol'),                # Símbolo del protocolo
        'tvl': protocol.get('tvl'),                      # Valor total bloqueado (TVL)
        'chains': protocol.get('chains'),                # Chains soportadas
        'change_1h': protocol.get('change_1h'),          # Variación de valor en 1 hora
        'change_1d': protocol.get('change_1d'),          # Variación de valor en 1 día
        'change_7d': protocol.get('change_7d'),          # Variación de valor en 7 días
    }
    extracted_data.append(protocol_info)

# Convertir a DataFrame para visualizar mejor la información
df = pd.DataFrame(extracted_data)

# Mostrar el DataFrame
df.head(10)

,name,symbol,tvl,chains,change_1h,change_1d,change_7d
0,Binance CEX,-,1.324858e+11,"[Bitcoin, Ethereum, Binance, Solana, Tron, Dog...",0.495652,1.839081,20.614670
1,Lido,LDO,3.110126e+10,"[Ethereum, Solana, Moonbeam, Moonriver, Terra]",0.069639,-0.921101,30.138609
2,Bitfinex,-,2.343860e+10,"[Bitcoin, Ethereum, Tron, Solana, Avalanche, F...",-0.011291,2.723769,21.600567
3,OKX,-,2.231087e+10,"[Bitcoin, Ethereum, Solana, Tron, Litecoin, Av...",0.260890,1.510107,20.080589
4,Robinhood,-,1.714986e+10,"[Bitcoin, Ethereum, Avalanche, Polygon]",0.392500,1.655439,25.440859
5,AAVE V3,AAVE,1.577613e+10,"[Ethereum, Arbitrum, Avalanche, Polygon, Base,...",0.071673,0.446763,26.661759
6,EigenLayer,-,1.347994e+10,[Ethereum],0.147305,-0.311307,30.345506
7,Bybit,-,1.294860e+10,"[Bitcoin, Ethereum, Solana, Binance, Tron, Rip...",0.306041,1.831185,23.104334
8,WBTC,-,1.208996e+10,[Bitcoin],0.541847,3.236720,19.644491
9,Arbitrum Bridge,ARB,9.086805e+09,[Ethereum],-1.278281,-2.883214,14.165935


In [4]:
main_url = "https://api.llama.fi/v2/historicalChainTvl/"

chains = ['Ethereum', 'Solana', 'BSC', 'Base', 'Tron']

dfs = {}

for chain in chains:
    api_url = main_url + chain
    response = requests.get(api_url)
    data = response.json()
    df = pd.DataFrame(data)

    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df.rename(columns={'tvl': chain}, inplace=True)

    dfs[chain] = df

df = pd.concat(dfs.values(), axis=1)
df 

,Ethereum,Solana,BSC,Base,Tron
date,,,,,
2018-04-27,4.040779e+05,NaN,NaN,NaN,NaN
2018-04-28,4.005851e+05,NaN,NaN,NaN,NaN
2018-04-29,4.292139e+05,NaN,NaN,NaN,NaN
2018-04-30,4.216512e+05,NaN,NaN,NaN,NaN
2018-05-01,4.361950e+05,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-11-07,5.346885e+10,6.805781e+09,4.694379e+09,2.697151e+09,6.722486e+09
2024-11-08,5.498260e+10,7.097521e+09,4.763689e+09,2.799963e+09,6.749441e+09
2024-11-09,5.694556e+10,7.162777e+09,4.911051e+09,2.885722e+09,6.734410e+09


In [5]:
import plotly.express as px

fig = px.line(df, x=df.index, y=df.columns, title="Chain TVL")
fig.update_layout(template='plotly_dark')

fig.show()

In [6]:
main_url = "https://yields.llama.fi/pools"

pools = requests.get(main_url)
pools = pools.json()
pools
if pools['status'] == 'success':
    # Extraer los datos y convertirlos a un DataFrame
    data = pools['data']
    df = pd.DataFrame(data)
    
    # Mostrar las primeras filas para verificar
    #print(df.head())
else:
    print("Error en la petición")

df.head(5)
# data = []

# # Extraer los datos específicos para cada protocolo
# for chain in pools:
#     protocol_info = {
#         'chain': pools.get('chain'),                    # Nombre del protocolo
#         'project': pools.get('project')        # Variación de valor en 7 días
#     }
#     data.append(protocol_info)

# # Convertir a DataFrame para visualizar mejor la información
# df = pd.DataFrame(data)

# # Mostrar el DataFrame
# df.head()

,chain,project,symbol,tvlUsd,apyBase,apyReward,apy,rewardTokens,pool,apyPct1D,...,sigma,count,outlier,underlyingTokens,il7d,apyBase7d,apyMean30d,volumeUsd1d,volumeUsd7d,apyBaseInception
0,Ethereum,lido,STETH,30945487389,3.05700,NaN,3.05700,None,747c1d2a-c668-4682-b9f9-296708a3dd90,0.13400,...,0.05068,895,False,[0x0000000000000000000000000000000000000000],NaN,NaN,2.96977,NaN,NaN,NaN
1,Solana,save,MANIFESTO,7060088132,0.00000,0.0,0.00000,[],ddc5900e-cbd3-491b-bbd3-cdb9a26b4ee4,0.00000,...,0.00000,6,False,[StaccN8ycAamAmZgijj9B7wKHwUEF17XN3vrNx1pQ6Z],NaN,NaN,0.00000,NaN,NaN,NaN
2,Ethereum,ether.fi-stake,WEETH,5732335953,3.02435,NaN,3.02435,None,46bd2bdf-6d92-4066-b482-e885ee172264,0.08286,...,0.03647,160,False,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2],NaN,3.01571,2.83864,NaN,NaN,NaN
3,Ethereum,binance-staked-eth,WBETH,4555820391,2.79699,NaN,2.79699,None,80b8bf92-b953-4c20-98ea-c9653ef2bb98,-0.04344,...,0.02989,513,False,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2],NaN,NaN,2.88972,NaN,NaN,NaN
4,Ethereum,rocket-pool,RETH,3992831659,2.94295,NaN,2.94295,None,d4b3c522-6127-4b89-bedf-83641cdcd2eb,0.01570,...,0.03997,663,False,[0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2],NaN,NaN,2.64126,NaN,NaN,NaN


In [7]:
data_base = [pool for pool in pools['data'] if pool['chain'] == 'Base']
df_base = pd.DataFrame(data_base)
df_base

,chain,project,symbol,tvlUsd,apyBase,apyReward,apy,rewardTokens,pool,apyPct1D,...,sigma,count,outlier,underlyingTokens,il7d,apyBase7d,apyMean30d,volumeUsd1d,volumeUsd7d,apyBaseInception
0,Base,origin-ether,SUPEROETHB,515307945,NaN,NaN,10.26280,None,f388573e-5c0f-4dac-9f70-116a4aabaf17,-0.12210,...,0.10174,62,False,[0x4200000000000000000000000000000000000006],NaN,NaN,11.02843,NaN,NaN,None
1,Base,aerodrome-v1,USDC-AERO,162703973,NaN,47.72157,47.72157,[0x940181a94A35A4569E4529A3CDfB74e38FD98631],d19fdacb-e7ce-400c-b62d-d195fe4a84a9,-1.46536,...,1.65721,307,True,"[0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913, 0...",-1.12986,NaN,51.84678,NaN,NaN,None
2,Base,aave-v3,WEETH,95081955,0.02283,NaN,0.02283,None,f0131970-afac-4835-b22c-520f192e01d5,-0.00008,...,0.01372,162,False,[0x04C0599Ae5A44757c0af6F9eC3b93da8976c150A],NaN,NaN,0.03040,NaN,NaN,None
3,Base,extra-finance,USDC-AERO,83430907,62.12134,NaN,62.12134,None,a8a60a49-4a0d-4ba4-93ec-2b3006a71b2d,NaN,...,4.09595,275,True,"[0x833589fcd6edb6e08f4c7c32d4f71b54bda02913, 0...",NaN,NaN,65.92422,NaN,NaN,None
4,Base,morpho-blue,MWETH,79650105,0.82181,0.00000,0.82181,[],e41c04d6-53b7-4e36-8de0-edda6f627103,0.08116,...,0.03637,145,False,[0x4200000000000000000000000000000000000006],NaN,NaN,0.93911,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,Base,aerodrome-v1,ALB-WETH,10068,NaN,0.00000,0.00000,[],290d4d19-5ba3-4e2a-945e-650d002c2bfa,0.00000,...,0.00000,5,False,"[0x1dd2d631c92b1aCdFCDd51A0F7145A50130050C4, 0...",-0.48614,NaN,0.00000,NaN,NaN,None
953,Base,baseswap-v2,WETH-ARX,10051,5.21549,79.04898,84.26447,"[0xd5046b976188eb40f6de40fb527f89c05b323385, 0...",3d1671f2-9920-4a83-82fb-f3b02d8e7f37,-0.02612,...,1.33007,273,True,"[0x4200000000000000000000000000000000000006, 0...",NaN,6.69518,76.37582,844.84072,7612.57616,None
954,Base,beefy,SFRXETH-AERO,10021,NaN,NaN,19.26968,None,0d38caef-e410-4ad5-b3dd-33ab5374d7e3,-0.39901,...,2.58864,211,True,"[0x1f55a02A049033E3419a8E2975cF3F572F4e6E9A, 0...",NaN,NaN,12.05549,NaN,NaN,None
955,Base,aerodrome-v1,AERO-DOG,10011,NaN,46.63188,46.63188,[0x940181a94A35A4569E4529A3CDfB74e38FD98631],3fb9194a-eb16-4480-8f15-fe6adb0b9969,NaN,...,2.24169,227,True,"[0x940181a94A35A4569E4529A3CDfB74e38FD98631, 0...",-0.46943,NaN,45.52778,NaN,NaN,None


In [8]:
baseUrl = 'https://api.llama.fi'

protocols = requests.get(baseUrl + '/protocols')

data = protocols.json()

#data = pools['data']
df = pd.DataFrame(data)
df.head(5)


,id,name,address,symbol,url,description,chain,logo,audits,audit_note,...,assetToken,oraclesByChain,pool2,stablecoins,misrepresentedTokens,language,tokensExcludedFromParent,deadUrl,deadFrom,rugged
0,2269,Binance CEX,None,-,https://www.binance.com,Binance is a cryptocurrency exchange which is ...,Multi-Chain,https://icons.llama.fi/binance-cex.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,182,Lido,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,https://lido.fi/,Liquid staking for Ethereum and Polygon. Daily...,Multi-Chain,https://icons.llama.fi/lido.png,2,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2275,Bitfinex,None,-,https://www.bitfinex.com,Bitfinex facilitates a graphical trading exper...,Multi-Chain,https://icons.llama.fi/bitfinex.png,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2272,OKX,None,-,https://www.okx.com,"OKX, formerly known as OKEx, is a Seychelles-b...",Multi-Chain,https://icons.llama.fi/okx.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3732,Robinhood,None,-,https://robinhood.com,Democratizing finance for all. Crypto trading:...,Multi-Chain,https://icons.llama.fi/robinhood.jpg,0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.columns

Index(['id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
       'logo', 'audits', 'audit_note', 'gecko_id', 'cmcId', 'category',
       'chains', 'module', 'twitter', 'forkedFrom', 'oracles', 'listedAt',
       'methodology', 'slug', 'tvl', 'chainTvls', 'change_1h', 'change_1d',
       'change_7d', 'tokenBreakdowns', 'mcap', 'referralUrl', 'treasury',
       'audit_links', 'openSource', 'governanceID', 'github', 'hallmarks',
       'parentProtocol', 'wrongLiquidity', 'staking', 'assetToken',
       'oraclesByChain', 'pool2', 'stablecoins', 'misrepresentedTokens',
       'language', 'tokensExcludedFromParent', 'deadUrl', 'deadFrom',
       'rugged'],
      dtype='object')

In [10]:
df[['name', 'symbol', 'chain', 'category', 'tvl', 'chainTvls', 'change_1h','change_1d', 'change_7d']]

,name,symbol,chain,category,tvl,chainTvls,change_1h,change_1d,change_7d
0,Binance CEX,-,Multi-Chain,CEX,1.324858e+11,"{'Fantom': 0, 'Algorand': 100240642.02427037, ...",0.495652,1.839081,20.614670
1,Lido,LDO,Multi-Chain,Liquid Staking,3.110126e+10,"{'Terra': 0, 'Ethereum': 31082534059.43622, 'M...",0.069639,-0.921101,30.138609
2,Bitfinex,-,Multi-Chain,CEX,2.343860e+10,"{'Fantom': 8940484.858054236, 'Litecoin': 7059...",-0.011291,2.723769,21.600567
3,OKX,-,Multi-Chain,CEX,2.231087e+10,"{'Aptos': 0, 'Ripple': 153939643.44188774, 'Li...",0.260890,1.510107,20.080589
4,Robinhood,-,Multi-Chain,CEX,1.714986e+10,"{'Polygon': 9120.741734168414, 'Ethereum': 552...",0.392500,1.655439,25.440859
...,...,...,...,...,...,...,...,...,...
5098,Common Wealth,WLTH,Base,Launchpad,0.000000e+00,"{'Base': 0, 'Base-staking': 2680887.6611139374...",NaN,NaN,NaN
5099,Ocean Point,BST,Ethereum,Farm,0.000000e+00,"{'Ethereum': 0, 'Ethereum-staking': 8553652.70...",NaN,NaN,NaN
5100,Pinjam Labs,PINKAV,Kava,Lending,-4.237372e+02,"{'Kava-staking': 2435.9941149975116, 'Kava-bor...",0.517763,NaN,NaN
5101,Market,-,Multi-Chain,Lending,-9.167591e+02,"{'Fantom': 0, 'Fantom-borrowed': 0, 'Polygon-b...",-0.353169,NaN,NaN


In [11]:
baseUrl = 'https://api.llama.fi/v2/historicalChainTvl/'

chain = 'Ethereum'

chaintv = requests.get(baseUrl + chain)

data = chaintv.json()

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'], unit='s')
df.set_index('date', inplace=True)
df.rename(columns={'tvl': 'Ethereum'}, inplace=True)
df.head(5)

,Ethereum
date,
2018-04-27,404077.858810
2018-04-28,400585.099213
2018-04-29,429213.933200
2018-04-30,421651.156158
2018-05-01,436194.969783


In [12]:
df['%1D'] = df['Ethereum'].pct_change(1) * 100    # Cambio diario
df['%7D'] = df['Ethereum'].pct_change(7) * 100    # Cambio semanal
df['%30D'] = df['Ethereum'].pct_change(30) * 100  # Cambio mensual
df.tail(5)

,Ethereum,%1D,%7D,%30D
date,,,,
2024-11-07,5.346885e+10,5.188339,6.494746,17.399801
2024-11-08,5.498260e+10,2.831077,13.294761,23.584228
2024-11-09,5.694556e+10,3.570141,17.630848,27.311914
2024-11-10,5.906511e+10,3.722074,23.152782,31.082833
2024-11-11,5.894129e+10,-0.209641,22.879672,28.000266


In [13]:
baseUrl = 'https://api.llama.fi/v2/historicalChainTvl/'

chain = 'Solana'

chaintv = requests.get(baseUrl + chain)

data = chaintv.json()

df_sol = pd.DataFrame(data)
df_sol['date'] = pd.to_datetime(df_sol['date'], unit='s')
df_sol.set_index('date', inplace=True)
df_sol.rename(columns={'tvl': 'Solana'}, inplace=True)
df_sol.head(5)

,Solana
date,
2021-03-18,1.489888e+08
2021-03-19,1.532043e+08
2021-03-20,1.476909e+08
2021-03-21,1.519353e+08
2021-03-22,1.529811e+08


In [14]:
df_sol['%1D'] = df_sol['Solana'].pct_change(1) * 100    # Cambio diario
df_sol['%7D'] = df_sol['Solana'].pct_change(7) * 100    # Cambio semanal
df_sol['%30D'] = df_sol['Solana'].pct_change(30) * 100  # Cambio mensual
df_sol.tail(5)

,Solana,%1D,%7D,%30D
date,,,,
2024-11-07,6.805781e+09,3.087708,8.490645,25.166410
2024-11-08,7.097521e+09,4.286651,16.846473,31.361817
2024-11-09,7.162777e+09,0.919424,17.802866,30.541069
2024-11-10,7.279920e+09,1.635434,21.775071,30.207543
2024-11-11,7.712423e+09,5.941044,29.383535,33.018494


In [15]:
def get_chain_data(chain):
    baseUrl = 'https://api.llama.fi/v2/historicalChainTvl/'
    chaintv = requests.get(baseUrl + chain).json()
    
    # Crear DataFrame y procesar
    df = pd.DataFrame(chaintv)
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df.rename(columns={'tvl': chain}, inplace=True)
    
    # Calcular cambios porcentuales
    df['%1D'] = df[chain].pct_change(1) * 100
    df['%7D'] = df[chain].pct_change(7) * 100
    df['%30D'] = df[chain].pct_change(30) * 100
    
    # Seleccionar la última fila de cambios porcentuales
    return df[['%1D', '%7D', '%30D']].iloc[-1]


chains = ['Ethereum', 'Solana', 'Arbitrum', 'Linea', 'Sui']
# Obtener los últimos cambios porcentuales para Ethereum y Solana

df_tvl = []
for chain in chains:
    chain_changes = get_chain_data(chain)
    df = pd.DataFrame({chain:chain_changes}).T
    df_tvl.append(df)

df = pd.concat(df_tvl)
df

,%1D,%7D,%30D
Ethereum,-0.209641,22.879672,28.000266
Solana,5.941044,29.383535,33.018494
Arbitrum,1.622656,20.804901,18.881317
Linea,-0.147768,22.944980,18.460064
Sui,2.498006,38.823402,26.011164


In [16]:
chain = 'ethereum'
baseUrl = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
chain_v = requests.get(baseUrl)
data = chain_v.json()

# Extrae los datos de interés, filtrando por la cadena específica si está en 'chains'
protocol_data = []
for protocol in data['protocols']:
    # Solo incluye los protocolos que estén en la cadena especificada
    if chain.capitalize() in protocol.get('chains', []):
        protocol_data.append({
            'name': protocol.get('name'),
            'chain': chain.capitalize(),
            'total24h': protocol.get('total24h'),
            'total48hto24h': protocol.get('total48hto24h'),
            'total7d': protocol.get('total7d'),
            'total30d': protocol.get('total30d'),
            'total1y': protocol.get('total1y'),
            'change_1d': protocol.get('change_1d'),
            'change_7d': protocol.get('change_7d'),
            'change_1m': protocol.get('change_1m')
        })

# Crea el DataFrame
df = pd.DataFrame(protocol_data)

# Muestra el DataFrame en formato tabular
df

,name,chain,total24h,total48hto24h,total7d,total30d,total1y,change_1d,change_7d,change_1m
0,Curve DEX,Ethereum,213641717,202915959,1.929081e+09,5.496287e+09,8.511246e+10,5.29,224.04,83.00
1,Balancer V1,Ethereum,503625,-19064297,-5.517357e+07,-6.842287e+07,6.493604e+08,-102.64,108.91,294.27
2,SushiSwap,Ethereum,11241859,13150666,6.246690e+07,2.068441e+08,6.059003e+09,-14.51,6.65,177.42
3,KyberSwap Classic,Ethereum,0,0,NaN,NaN,4.350812e+07,NaN,NaN,NaN
4,DODO AMM,Ethereum,97393744,77246028,6.560933e+08,2.686553e+09,4.468287e+10,26.08,9.01,13.30
...,...,...,...,...,...,...,...,...,...,...
67,Basin Exchange,Ethereum,166599,26043,2.811160e+05,8.734200e+05,5.614896e+07,539.71,6347.33,77.06
68,Maverick V2,Ethereum,68562393,58827607,4.480650e+08,1.707819e+09,3.131442e+09,16.55,5.30,439.45
69,Apex DeFi,Ethereum,1829,3376,1.223700e+04,4.917900e+04,1.506340e+05,-45.82,107.13,-11.98
70,Raindex,Ethereum,4417,5541,7.592800e+04,1.714830e+05,1.620504e+06,-20.29,46.65,316.31


In [17]:
# import requests

# url = "https://api.coingecko.com/api/v3/coins/markets"

# params = {
#     #'ids': 'bitcoin',
#     'vs_currencies':'USD'
# }

# headers = {'x-cg-demo-api-key':''}

# response = requests.get(url, params = params)
# response,json()

# # if response.status_code == 200:
# #          data = response.json()
# #          Ethereum_price = data['bitcoin']['usd']
# #          print(f'The price of bitcoin in USD is ${Ethereum_price}')
# # else:
# #          print('Failed to retrieve data from the API')

# # data = response.json()
# # data

In [18]:
from pycoingecko import CoinGeckoAPI

In [19]:
cg = CoinGeckoAPI()

In [62]:
coinList = cg.get_coins_list()
df_coin = pd.DataFrame.from_dict(coinList)

# df_coin[df_coin['id'] == 'bitcoin']
# df_coin[df_coin['id'] == 'ethereum']
df_coin[df_coin['id'] == 'DOG']

,id,symbol,name


In [21]:
coins = ['bitcoin','ethereum','dopex']

getcoin = cg.get_supported_vs_currencies()
vsCurrencies = ['usd', 'eur']

simplePrice = cg.get_price(ids=coins, vs_currencies='usd')
simplePrice

{'bitcoin': {'usd': 84192},
 'dopex': {'usd': 11.26},
 'ethereum': {'usd': 3253.03}}

In [22]:
complexPriceRequest = cg.get_price(ids = coins, 
                        vs_currencies = vsCurrencies, 
                        include_market_cap = True,
                        include_24hr_vol = True,
                        include_24hr_change = True,
                        include_last_updated_at = True)
complexPriceRequest = pd.DataFrame(complexPriceRequest)
complexPriceRequest.T

,usd,usd_market_cap,usd_24h_vol,usd_24h_change,eur,eur_market_cap,eur_24h_vol,eur_24h_change,last_updated_at
bitcoin,84192.00,1.668085e+12,9.834589e+10,5.595166,79149.00,1.567971e+12,9.245566e+10,6.403400,1.731340e+09
dopex,11.26,0.000000e+00,1.058219e+05,0.418761,10.58,0.000000e+00,9.947075e+04,1.174028,1.731340e+09
ethereum,3253.03,3.914325e+11,5.185264e+10,1.471069,3058.20,3.679399e+11,4.874703e+10,2.247737,1.731340e+09


In [23]:
assetplatformlist = cg.get_asset_platforms()
assetplatformlist = pd.DataFrame(assetplatformlist)
assetplatformlist

,id,chain_identifier,name,shortname,native_coin_id,image
0,valobit,NaN,Valobit,,valobit,{'thumb': 'https://coin-images.coingecko.com/a...
1,arbitrum-nova,42170.0,Arbitrum Nova,,ethereum,{'thumb': 'https://coin-images.coingecko.com/a...
2,factom,NaN,Factom,,factom,"{'thumb': None, 'small': None, 'large': None}"
3,monad,NaN,Monad,,monad,{'thumb': 'https://coin-images.coingecko.com/a...
4,fraxtal,252.0,Fraxtal,,fraxtal,{'thumb': 'https://coin-images.coingecko.com/a...
...,...,...,...,...,...,...
335,chia,NaN,Chia,,chia,{'thumb': 'https://coin-images.coingecko.com/a...
336,klay-token,8217.0,Kaia,,kaia,{'thumb': 'https://coin-images.coingecko.com/a...
337,sei-v2,1329.0,Sei V2,,wrapped-sei,{'thumb': 'https://coin-images.coingecko.com/a...
338,apechain,33139.0,ApeChain,,apecoin,{'thumb': 'https://coin-images.coingecko.com/a...


In [24]:
assetplatformlist[assetplatformlist['id'] == 'binance-smart-chain']

,id,chain_identifier,name,shortname,native_coin_id,image
144,binance-smart-chain,56.0,BNB Smart Chain,BSC,binancecoin,{'thumb': 'https://coin-images.coingecko.com/a...


In [25]:
AVAXpriceBSC = cg.get_token_price(id = 'binance-smart-chain',
                                  contract_addresses='0x1ce0c2827e2ef14d5c4f' + '29a091d735a204794041',
                                  vs_currencies = 'usd')
AVAXpriceBSC

{'0x1ce0c2827e2ef14d5c4f29a091d735a204794041': {'usd': 32.58}}

In [26]:
coinCategorieList = pd.DataFrame(cg.get_coins_categories_list())
coinCategorieList

,category_id,name
0,8bit-chain-ecosystem,8Bit Chain Ecosystem
1,aave-tokens,Aave Tokens
2,account-abstraction,Account Abstraction
3,adidas-ecosystem,Adidas Ecosystem
4,adventure-games,Adventure Games
...,...,...
549,zklink-nova-ecosystem,zkLink Nova Ecosystem
550,zksync-ecosystem,ZkSync Ecosystem
551,zodiac-themed,Zodiac-Themed
552,zoo-themed,Zoo-Themed


In [27]:
coinCategoriesData = pd.DataFrame(cg.get_coins_categories(order = 'market_cap_desc'))
#coinCategoriesData.id.head(20)
coinCategoriesData[coinCategoriesData['id'] == 'meme-token']


,id,name,market_cap,market_cap_change_24h,content,top_3_coins_id,top_3_coins,volume_24h,updated_at
15,meme-token,Meme,1.030767e+11,28.961769,"Meme coins derive their relevance from memes, ...","[dogecoin, shiba-inu, pepe]",[https://coin-images.coingecko.com/coins/image...,5.288162e+10,2024-11-11T15:40:19.419Z


In [28]:
coinsMktDataByIds = cg.get_coins_markets(vs_currency = 'usd',ids = coins)
coinsMktDataByIds = pd.DataFrame(coinsMktDataByIds)
coinsMktDataByIds

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,83816.00,1.661169e+12,1.0,1763558225812,99193718362,84192.00,...,2.100000e+07,21000000.0,84022.00,0.22775,2024-11-11T15:36:14.845Z,67.810000,124092.25320,2013-07-06T00:00:00.000Z,None,2024-11-11T15:46:04.881Z
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,3230.22,3.908729e+11,2.0,390872946616,51790548263,3253.03,...,1.204238e+08,NaN,4878.26,-33.31198,2021-11-10T14:24:19.604Z,0.432979,751256.62182,2015-10-20T00:00:00.000Z,"{'times': 50.49391312891621, 'currency': 'btc'...",2024-11-11T15:46:03.535Z
2,dopex,dpx,Dopex,https://coin-images.coingecko.com/coins/images...,11.24,0.000000e+00,NaN,5629677,105811,11.31,...,5.000000e+05,500000.0,4215.41,-99.73427,2022-01-15T20:54:31.557Z,0.004758,235324.95422,2024-09-17T05:10:21.474Z,None,2024-11-11T15:46:06.087Z


In [29]:
coinsMktDataByCategory = cg.get_coins_markets(vs_currency = 'usd',category = 'meme-token',
                                    order = 'volume_desc')

coinsMktDataByCategory = pd.DataFrame(coinsMktDataByCategory)
coinsMktDataByCategory.columns

Index(['id', 'symbol', 'name', 'image', 'current_price', 'market_cap',
       'market_cap_rank', 'fully_diluted_valuation', 'total_volume',
       'high_24h', 'low_24h', 'price_change_24h',
       'price_change_percentage_24h', 'market_cap_change_24h',
       'market_cap_change_percentage_24h', 'circulating_supply',
       'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
       'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi',
       'last_updated'],
      dtype='object')

In [67]:
df = coinsMktDataByCategory[['name','symbol', 'current_price', 'market_cap', 'price_change_24h']]
#df =df[df['market_cap'] >= 200000000]
df = df.loc[df['market_cap'] >=500000000 ]
df


,name,symbol,current_price,market_cap,price_change_24h
0,Dogecoin,doge,0.308121,4.543851e+10,6.215700e-02
1,Shiba Inu,shib,0.000028,1.620300e+10,1.460000e-06
2,Pepe,pepe,0.000012,5.196980e+09,-1.010740e-07
3,dogwifhat,wif,3.030000,3.049151e+09,3.317760e-01
5,FLOKI,floki,0.000181,1.764233e+09,5.530000e-06
6,Neiro,neiro,0.002432,1.023852e+09,-2.019840e-04
7,Bonk,bonk,0.000026,1.823797e+09,1.325070e-07
8,BOOK OF MEME,bome,0.009252,6.444088e+08,-3.316630e-04
11,Notcoin,not,0.007382,7.562932e+08,-5.200472e-04
12,cat in a dogs world,mew,0.010956,9.717569e+08,3.732000e-04


In [77]:
import plotly.express as px

fig = px.pie(
    df, 
    values='market_cap', 
    names='symbol', 
    title="Market Cap Memecoins",
    color_discrete_sequence=px.colors.qualitative.Pastel  # Colores suaves para mejor visualización
)
fig.update_layout(
    showlegend=True,
    legend_title_text='Criptomonedas',
    margin=dict(t=50, b=50, l=25, r=25),
    template='plotly_dark')


fig.show()

In [56]:
coinsMktDataById = cg.get_coins_markets(vs_currency = 'usd',
                                      ids = ['dogecoin','shiba-inu', 'pepe', 'dogwifhat'],
                                      price_change_percentage = '1h,24h,7d',
                                      sparkline = True)

coinmk = pd.DataFrame(coinsMktDataById)
coinmk

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,sparkline_in_7d,price_change_percentage_1h_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_7d_in_currency
0,dogecoin,doge,Dogecoin,https://coin-images.coingecko.com/coins/images...,0.308261,45214502755,6,45222995860,23124663012,0.310483,...,2021-05-08T05:08:23.458Z,8.690000e-05,3.575054e+05,2015-05-06T00:00:00.000Z,None,2024-11-11T15:51:20.421Z,"{'price': [0.15686792582182887, 0.157171013788...",6.288947,25.094945,97.820220
1,shiba-inu,shib,Shiba Inu,https://coin-images.coingecko.com/coins/images...,0.000028,16273111615,11,27616081856,7848062193,0.000028,...,2021-10-28T03:54:55.568Z,5.636600e-11,4.975227e+07,2020-11-28T11:26:25.838Z,None,2024-11-11T15:51:24.914Z,"{'price': [1.7182206501791547e-05, 1.712980194...",7.662356,6.487519,63.189792
2,pepe,pepe,Pepe,https://coin-images.coingecko.com/coins/images...,0.000012,5203687727,29,5203687727,5559755429,0.000013,...,2024-05-27T08:30:07.709Z,5.514200e-08,2.231089e+04,2023-04-18T02:14:41.591Z,None,2024-11-11T15:51:23.772Z,"{'price': [8.300056599766103e-06, 8.2638163140...",2.717117,-1.495025,51.950081


In [52]:
import plotly.express as px 

fig = px.bar(coinmk, x='symbol', y='price_change_percentage_7d_in_currency')
fig.update_layout(template='plotly_dark')
fig.show()


In [103]:
mktSpecificData = cg.get_coin_ticker_by_id(id='bitcoin',
                         exchange_id='aave',
                         page=1,
                         depth = True)
mktSpecificData = pd.DataFrame(mktSpecificData)

In [90]:
mktSpecificDataFrame = pd.DataFrame.from_dict(mktSpecificData['tickers']
                                  ).sort_values('base').reset_index(drop=True)
mktSpecificDataFrame

,base,target,market,last,volume,cost_to_move_up_usd,cost_to_move_down_usd,converted_last,converted_volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,is_stale,trade_url,token_info_url,coin_id,target_coin_id
0,ADA,BTC,"{'name': 'WhiteBIT', 'identifier': 'whitebit',...",0.000007,3.640728e+07,3.624179e+05,5.718682e+05,"{'btc': 1.0, 'eth': 25.721519, 'usd': 84863}","{'btc': 254.997, 'eth': 6559, 'usd': 21639649}",green,0.028555,2024-11-11T18:32:59+00:00,2024-11-11T18:32:59+00:00,2024-11-11T18:32:59+00:00,False,False,https://whitebit.com/trade/ADA_BTC,None,cardano,bitcoin
1,AVAX,BTC,"{'name': 'WhiteBIT', 'identifier': 'whitebit',...",0.000403,1.020147e+05,4.195189e+04,4.282440e+04,"{'btc': 1.0, 'eth': 25.719673, 'usd': 84810}","{'btc': 41.147637, 'eth': 1058, 'usd': 3489729}",green,0.012484,2024-11-11T18:31:13+00:00,2024-11-11T18:31:13+00:00,2024-11-11T18:31:13+00:00,False,False,https://whitebit.com/trade/AVAX_BTC,None,avalanche-2,bitcoin
2,BCH,BTC,"{'name': 'Binance', 'identifier': 'binance', '...",0.005173,8.499260e+03,5.712744e+04,9.008126e+04,"{'btc': 1.0, 'eth': 25.711864, 'usd': 84550}","{'btc': 45.39182, 'eth': 1167, 'usd': 3837878}",green,0.019346,2024-11-11T18:24:25+00:00,2024-11-11T18:24:25+00:00,2024-11-11T18:25:10+00:00,False,False,https://www.binance.com/en/trade/BCH_BTC?ref=3...,None,bitcoin-cash,bitcoin
3,BNB,BTC,"{'name': 'Binance', 'identifier': 'binance', '...",0.007466,6.343686e+04,8.203414e+05,6.999110e+05,"{'btc': 1.0, 'eth': 25.722733, 'usd': 84828}","{'btc': 486.299, 'eth': 12509, 'usd': 41251721}",green,0.013403,2024-11-11T18:32:19+00:00,2024-11-11T18:32:19+00:00,2024-11-11T18:32:19+00:00,False,False,https://www.binance.com/en/trade/BNB_BTC?ref=3...,None,binancecoin,bitcoin
4,BTC,USDT,"{'name': 'BVOX', 'identifier': 'bitvenus_spot'...",84484.850000,8.755338e+04,1.058311e+07,1.074358e+07,"{'btc': 1.001328, 'eth': 25.699617, 'usd': 84608}","{'btc': 84969, 'eth': 2180775, 'usd': 7179507463}",green,0.010012,2024-11-11T18:15:28+00:00,2024-11-11T18:15:28+00:00,2024-11-11T18:15:28+00:00,False,False,https://www.bitvenus.me/exchange/BTC/USDT,None,bitcoin,tether
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SOL,BTC,"{'name': 'Binance', 'identifier': 'binance', '...",0.002568,8.684844e+05,1.775411e+06,2.339397e+06,"{'btc': 1.0, 'eth': 25.700573, 'usd': 85038}","{'btc': 2262, 'eth': 58124, 'usd': 192321641}",green,0.013894,2024-11-11T18:40:03+00:00,2024-11-11T18:40:03+00:00,2024-11-11T18:40:03+00:00,False,False,https://www.binance.com/en/trade/SOL_BTC?ref=3...,None,solana,bitcoin
96,SOL,BTC,"{'name': 'OKX', 'identifier': 'okex', 'has_tra...",0.002568,5.556100e+04,4.441419e+05,3.125739e+05,"{'btc': 1.0, 'eth': 25.707107, 'usd': 84965}","{'btc': 144.912, 'eth': 3725, 'usd': 12312460}",green,0.027254,2024-11-11T18:39:56+00:00,2024-11-11T18:39:56+00:00,2024-11-11T18:39:56+00:00,False,False,https://www.okx.com/trade-spot/sol-btc,None,solana,bitcoin
97,SOL,BTC,"{'name': 'WhiteBIT', 'identifier': 'whitebit',...",0.002566,5.880945e+04,3.674168e+05,4.850840e+05,"{'btc': 1.0, 'eth': 25.711363, 'usd': 84927}","{'btc': 150.93, 'eth': 3881, 'usd': 12818008}",green,0.014024,2024-11-11T18:38:12+00:00,2024-11-11T18:38:12+00:00,2024-11-11T18:38:12+00:00,False,False,https://whitebit.com/trade/SOL_BTC,None,solana,bitcoin
98,SUI,BTC,"{'name': 'Binance', 'identifier': 'binance', '...",0.000037,6.632799e+06,2.029914e+05,2.442382e+05,"{'btc': 1.0, 'eth': 25.700573, 'usd': 85038}","{'btc': 256.164, 'eth': 6584, 'usd': 21783718}",green,0.026838,2024-11-11T18:40:08+00:00,2024-11-11T18:40:08+00:00,2024-11-11T18:40:08+00:00,False,False,https://www.binance.com/en/trade/SUI_BTC?ref=3...,None,sui,bitcoin


In [94]:
dailyHistoricalData = cg.get_coin_market_chart_by_id(id = 'bitcoin', 
                               vs_currency = 'usd',
                               days = '100')

In [93]:
hourlyHistoricalData = cg.get_coin_market_chart_by_id(id = 'bitcoin', 
                               vs_currency = 'usd',
                               days = 90)

hourlyHistoricalData

{'prices': [[1723576760855, 60586.49440610235],
  [1723580600816, 60700.017313229466],
  [1723583828309, 60634.62198998188],
  [1723586919348, 60930.70826903593],
  [1723591234790, 60520.521585843],
  [1723593759020, 60611.1178437017],
  [1723597859402, 60754.85111728312],
  [1723601831757, 60680.99542837812],
  [1723605626493, 60974.5246749954],
  [1723609330085, 60934.71745277854],
  [1723612585267, 61018.6692167453],
  [1723615259898, 60892.081560238315],
  [1723619324286, 60780.9965478888],
  [1723622720166, 60932.51558979862],
  [1723626758740, 60905.02264032649],
  [1723630878850, 61116.85624966809],
  [1723634510228, 60948.01211615021],
  [1723637563782, 61356.318193391344],
  [1723641448807, 60741.11900146548],
  [1723645109134, 58989.47034890381],
  [1723648758231, 59296.870497296506],
  [1723652400444, 59398.18261345918],
  [1723655945651, 59247.72206319977],
  [1723659128619, 59232.393107718366],
  [1723662731532, 58938.9576899662],
  [1723666094074, 58992.182451813904],
  [

In [95]:
globalDefiData = cg.get_global_decentralized_finance_defi()
globalDefiData

{'defi_market_cap': '89861784236.491955156783388484',
 'eth_market_cap': '400298980128.575442517412448024',
 'defi_to_eth_ratio': '22.4486667959105173050108712535626077861457544857714841',
 'trading_volume_24h': '12558612273.318052447070526168',
 'defi_dominance': '2.9477605559448324814442541368107329484629417565000481',
 'top_coin_name': 'Lido Staked Ether',
 'top_coin_defi_dominance': 36.294780002331436}

In [100]:
coinsMktDataById = cg.get_coins_markets(vs_currency = 'usd',
                                      ids = 'bitcoin',
                                      price_change_percentage = '1h,24h,7d',
                                      sparkline = True)
coinsMktDataById = pd.DataFrame(coinsMktDataById)
coinsMktDataById

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,sparkline_in_7d,price_change_percentage_1h_in_currency,price_change_percentage_24h_in_currency,price_change_percentage_7d_in_currency
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,86949,1720076079791,1,1826084364077,119529753282,87343,...,2024-11-11T20:51:47.909Z,67.81,127942.1838,2013-07-06T00:00:00.000Z,None,2024-11-11T21:01:55.223Z,"{'price': [67627.43829020744, 67939.3587094227...",0.522143,10.194187,28.424362


### Ojo array

In [140]:

baseUrl = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
response = requests.get(baseUrl)

if response.status_code == 200:
    data = response.json()
    # Extraer datos de los protocolos
    protocols = data.get('protocols', [])
    
    # Crear el DataFrame con el nombre y el volumen total de 24h de cada protocolo
    df = pd.DataFrame([{
        "name": protocol.get("name"),
        "total24h": protocol.get("total24h"),
        "total48hto24h": protocol.get("total48hto24h")
    } for protocol in protocols])
    
    print(df)
else:
    print("Error:", response.status_code)

                  name     total24h  total48hto24h
0            Curve DEX  214226124.0    202915959.0
1          Balancer V1     503625.0    -19064297.0
2            SushiSwap   11241859.0     13150666.0
3    KyberSwap Classic          0.0            0.0
4       Shell Protocol       6673.0         6220.0
..                 ...          ...            ...
586     Cables Finance         35.0         9053.0
587        DeepBook V3   22483379.0     15440930.0
588      Nabla Finance    1371822.0       147901.0
589               KYEX          7.0            0.0
590          Fluid DEX   68923842.0     37015724.0

[591 rows x 3 columns]


KeyError: 'data'